# HW10 Plane Crash

Name: Eric Wu

### Imports

In [1]:
#bigdir = '/Users/jkinser/Documents/CoursesKinser/CDSCardinal/Version5/'
#bigdir = '/home/jkinser/Documents/courses/CDSCardinal/Version5/'
bigdir = '/Users/eric/Documents/CDS465/'

pydir = bigdir + 'pysrc/'
datadir = bigdir + 'data/'
popdir = bigdir + 'population/'

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
import sys
import warnings
warnings.filterwarnings('ignore', category=FutureWarning) 

sys.path.append( pydir )
import Main
import detective as dtcv

E = Main.Realm(datadir)
E.ReadAll('/Users/eric/Documents/University/CDS465HW/HW10/planecrash.db')
len(E.persondf)

101916

# Identify missing persons

In [2]:
#import manifest and DNA excel files
manif = pd.read_excel('/Users/eric/Documents/University/CDS465HW/HW10/manifest.xlsx').drop(['level_0','index'],axis=1)
dna = pd.read_excel('/Users/eric/Documents/University/CDS465HW/HW10/dna.xlsx')

In [3]:
### Try getting pids and legal parents from social query using manifest

keep = ['pid','birth','knowndeath','firstname','lastname', 'gender', 'hoh','spouse', 'legaldad','legalmom', 'dometh','locv','loch']
temp_manif = pd.DataFrame(data=[], columns=keep)

for row in manif.itertuples():
    temp_manif = pd.concat([temp_manif, dtcv.SocialQuery(E, row.firstname, row.lastname)], ignore_index=True)

temp_manif = temp_manif.query(f'knowndeath == {temp_manif['knowndeath'].max()}').drop_duplicates() #get pids for those on the manifest AND filter for those who's known death was equivalent to the max

temp_manif

,pid,birth,knowndeath,firstname,lastname,gender,hoh,spouse,legaldad,legalmom,dometh,locv,loch,firstnameS,lastnameS,firstnameD,lastnameD,firstnameM,lastnameM
2,89676,7727,8200,Wanetta,Brey,F,0,NaN,NaN,NaN,0,1130.612725,675.235720,NaN,NaN,Dominick,Wesolowski,Joaquina,Wesolowski
3,93245,7874,8200,Rick,Desalvo,M,0,NaN,NaN,NaN,0,1043.292196,1131.402808,NaN,NaN,Hipolito,Desalvo,Eustolia,Desalvo
4,97696,8057,8200,Arlie,Fort,M,0,NaN,NaN,NaN,2,1077.729999,1144.727351,NaN,NaN,Miguel,Fort,Jacki,Fort
5,98712,8097,8200,Carmella,Fort,F,0,NaN,NaN,NaN,3,1077.729999,1144.727351,NaN,NaN,Miguel,Fort,Jacki,Fort
6,89148,7707,8200,Jacki,Fort,F,0,NaN,NaN,NaN,2,1077.729999,1144.727351,NaN,NaN,Kirk,McLendon,Regan,Turman
7,88794,7694,8200,Miguel,Fort,M,0,NaN,NaN,NaN,3,1077.729999,1144.727351,NaN,NaN,Sang,Fort,Sonya,Pompey
8,99761,8134,8200,Nelson,Fort,M,0,NaN,NaN,NaN,2,1077.729999,1144.727351,NaN,NaN,Miguel,Fort,Jacki,Fort
9,98964,8106,8200,Anneliese,Hickmon,F,0,NaN,NaN,NaN,0,573.100734,1298.895688,NaN,NaN,Hobert,Hickmon,Mari,Hickmon
10,89397,7717,8200,Hobert,Hickmon,M,0,NaN,NaN,NaN,0,573.100734,1298.895688,NaN,NaN,Terrell,Hickmon,Susana,Hickmon
11,99480,8124,8200,Hobert,Hickmon,M,0,NaN,NaN,NaN,0,573.100734,1298.895688,NaN,NaN,Hobert,Hickmon,Mari,Hickmon


Nally Quincy, M, 101489 may be our immigrant since his legal D&M don't show up in social query

Jacki and Miguel Fort are the parents of the Fort Family

Hobert and Mari Hickman are the parents of the Hickman Family

In [4]:
### Check Quincy Nally is the immigrant

dtcv.PortQuery(E, 'Quincy', 'Nally')

,firstname,lastname,immigdate,emigdate
8144,Quincy,Nally,8185,-1


In [5]:
### Cartesian join; elim mismatches of approx age and gender.

mnfdnadf = pd.merge(manif, dna, how='cross', suffixes=('_mnf', '_dna')).query('gender_mnf == gender_dna and approx_mnf == approx_dna')
mnfdnadf.drop(['gender_dna', 'approx_dna'],axis=1,inplace=True)

In [6]:
### Try matching DNA based off names
for row in manif.itertuples():
    jailrec = dtcv.JailQuery(E, row.firstname, row.lastname)
    print(pd.merge(jailrec, dna.drop(['mtdna', 'ystr'], axis=1), on='nstr') if len(jailrec) > 0 else f"No record found for {row.firstname} {row.lastname}")

  firstname lastname  start   end  \
0   Wanetta     Brey   8048  8113   

                                                                                                                 nstr  \
0  (12,14);(29,30);(9,10);(9,12);(14,15);(6,9);(9,13);(10,13);(17,19);(14,14);(14,18);(11,11);(12,21);(11,11);(24,26)   

  ystr                 mtdna  Unnamed: 0 gender approx  
0       1;3;w1;1;2;2;4;1;1;1          11      F  Adult  
No record found for Rick Desalvo
No record found for Arlie Fort
No record found for Carmella Fort
No record found for Jacki Fort
No record found for Miguel Fort
No record found for Nelson Fort
No record found for Anneliese Hickmon
No record found for Hobert Hickmon
No record found for Hobert Hickmon
No record found for Luise Hickmon
No record found for Mari Hickmon
No record found for Ashanti McGary
No record found for Quincy Nally
No record found for Shalom Spindler


Well now we for sure know one person's dna profile: Wanetta Brey 

In [7]:
### Remove rows that do not fit the criteria
fname, lname = 'Wanetta', 'Brey'
nstr_WB = '(12,14);(29,30);(9,10);(9,12);(14,15);(6,9);(9,13);(10,13);(17,19);(14,14);(14,18);(11,11);(12,21);(11,11);(24,26)'

drop_ndx = mnfdnadf.query('firstname == @fname and lastname == @lname and nstr != @nstr_WB').index
mnfdnadf = mnfdnadf.drop(drop_ndx)

drop_ndx = mnfdnadf.query('firstname != @fname and lastname != @lname and nstr == @nstr_WB').index
mnfdnadf = mnfdnadf.drop(drop_ndx)

In [8]:
### Check wedding records of the children
lnames = ['Hickmon', 'Fort']

for row in temp_manif.query('lastname in @lnames and birth < 8000')[['firstname', 'lastname']].itertuples():
    print('Wedding rec for', row.firstname, row.lastname)
    print(dtcv.WeddingQuery(E, row.firstname, row.lastname))

Wedding rec for Jacki Fort
Empty DataFrame
Columns: [date, firstH, lastH, firstW, lastW, divorce, spdie]
Index: []
Wedding rec for Miguel Fort
       date  firstH lastH firstW     lastW  divorce  spdie
index                                                     
76366  7923  Miguel  Fort  Jacki  McLendon       -1   8200
Wedding rec for Hobert Hickmon
       date  firstH    lastH firstW   lastW  divorce  spdie
index                                                      
76575  7933  Hobert  Hickmon  Betsy  Uresti       -1   8155
81879  8167  Hobert  Hickmon   Mari   Enger       -1   8200
Wedding rec for Mari Hickmon
Empty DataFrame
Columns: [date, firstH, lastH, firstW, lastW, divorce, spdie]
Index: []


The Fort Family may be the family where all 3 children are legal and bio children

Hobert Hickmon may have had children previously.

In [9]:
print(dtcv.WeddingQuery(E, 'Jacki', 'McLendon'))

print(dtcv.WeddingQuery(E, 'Betsy', 'Uresti'))

print(dtcv.WeddingQuery(E, 'Mari', 'Enger'))

       date  firstH lastH firstW     lastW  divorce  spdie
index                                                     
76366  7923  Miguel  Fort  Jacki  McLendon       -1   8200
       date  firstH    lastH firstW   lastW  divorce  spdie
index                                                      
76575  7933  Hobert  Hickmon  Betsy  Uresti       -1   8155
       date    firstH    lastH firstW  lastW  divorce  spdie
index                                                       
76238  7918  Laurence   Corman   Mari  Enger       -1   8154
81879  8167    Hobert  Hickmon   Mari  Enger       -1   8200


Jacki has only ever been married to Miguel so the in the Fort Family, all 3 children are bio and legal

The Hickmon family may have children from mothers Betsy, Mari and fathers Hobert, Laurence

In [10]:
dna.groupby('mtdna').count()

,Unnamed: 0,nstr,ystr,gender,approx
mtdna,,,,,
0;1;1;1;1;2;w1;1;1;1,1,1,0,1,1
1;0;0;2;2;2;3;1;1;1,1,1,1,1,1
1;1;0;2;1;2;1;0;1;1,1,1,1,1,1
1;1;1;1;1;2;7;1;1;1,1,1,0,1,1
1;1;3;1;1;2;0;1;1;1,3,3,1,3,3
1;1;4;1;1;2;5;1;1;1,1,1,1,1,1
1;2;1;1;2;2;w2;1;1;1,4,4,2,4,4
1;2;3;1;2;2;1;5;1;1,1,1,1,1,1
1;3;3;1;1;2;1;1;1;0,1,1,1,1,1


1;2;1;1;2;2;w2;1;1;1 is the mtdna of the Fort Family

1;1;3;1;1;2;0;1;1;1 is the mtdna of the Hickmon Family

In [11]:
### Remove rows that do not fit the criteria of Fort and mtdna_fort
lname = 'Fort'
mtdna_fort = '1;2;1;1;2;2;w2;1;1;1'

drop_ndx = mnfdnadf.query('lastname != @lname and mtdna == @mtdna_fort').index
mnfdnadf = mnfdnadf.drop(drop_ndx)

In [12]:
### Remove rows that do not fit the criteria of Hickmon and mtdna_hickmon
lname = 'Hickmon'
mtdna_hickmon = '1;1;3;1;1;2;0;1;1;1'

drop_ndx = mnfdnadf.query('lastname != @lname and mtdna == @mtdna_hickmon').index
mnfdnadf = mnfdnadf.drop(drop_ndx)

In [13]:
dna.groupby('ystr').count()

,Unnamed: 0,nstr,mtdna,gender,approx
ystr,,,,,
"15;13;22;29;15.0;14;17,18;13;10;12;22;11;22;14;11;21",1,1,1,1,1
"15;13;23;29;17.0;15;17,17;13;11;12;23;11;25;14;12;19",1,1,1,1,1
"15;13;25;31;17;15;16,16;13;10;13;23;11;21;14;11;21.0",1,1,1,1,1
"16;12;24;29;17.0;15;18,18;13;11;11;23;13;22;16;12;20,21",2,2,2,2,2
"16;13;24;29;18.0;14;15,18;13;10;12;22;13;23;14;12;20",3,3,3,3,3


ystr of 16;13;24;29;18.0;14;15,18;13;10;12;22;13;23;14;12;20 is likely the Fort Family; has 3M, 2F

ystr of 16;12;24;29;17.0;15;18,18;13;11;11;23;13;22;16;12;20,21 is likely the Hickmon Family; has 2M, 3F

In [14]:
### Remove rows that do not fit the criteria of Fort and ystr_fort
lname = 'Fort'
ystr_fort = '16;13;24;29;18.0;14;15,18;13;10;12;22;13;23;14;12;20'

drop_ndx = mnfdnadf.query('ystr == @ystr_fort and lastname != @lname').index # remove occurances of the Fort family ystr found in non Fort members
mnfdnadf = mnfdnadf.drop(drop_ndx)

In [15]:
### Remove rows that do not fit the criteria of Hickmon and ystr_hickmon
lname = 'Hickmon'
ystr_hickmon = '16;12;24;29;17.0;15;18,18;13;11;11;23;13;22;16;12;20,21'

drop_ndx = mnfdnadf.query('ystr == @ystr_hickmon and lastname != @lname').index
mnfdnadf = mnfdnadf.drop(drop_ndx)

In [16]:
### Dna check similarity in str !!! WARNING BAD CODE LOL
def parse_str_alleles(str_profile):
    cleaned_string = str_profile.replace(';', ',').replace('(', '').replace(')', '')
    alleles = [a.strip() for a in cleaned_string.split(',') if a.strip()]
    return alleles

dna2 = dna.copy()
dna2['Allele_List'] = dna2['nstr'].apply(parse_str_alleles)

def calculate_pct_similarity(alleles_a, alleles_b):
    if alleles_a is None or alleles_b is None:
        return 0.0
    min_len = min(len(alleles_a), len(alleles_b))
    matches = sum(1 for a, b in zip(alleles_a[:min_len], alleles_b[:min_len]) if a == b)
    total_comparisons = min_len
    if total_comparisons == 0:
        return 0.0
    return (matches / total_comparisons) * 100

allele_lists = dna2['Allele_List'].tolist()
num_profiles = len(allele_lists)
similarity_matrix_data = np.zeros((num_profiles, num_profiles))

for i in range(num_profiles):
    for j in range(num_profiles):
        if j >= i:
            score = calculate_pct_similarity(allele_lists[i], allele_lists[j])
            similarity_matrix_data[i, j] = score
            similarity_matrix_data[j, i] = score

similarity_matrix = pd.DataFrame(
    similarity_matrix_data,
    index=dna2.index,
    columns=dna2.index
)

pd.set_option('display.max_columns', None)
print("\nSTR DNA Similarity Matrix (Percentage):")
print(similarity_matrix)


STR DNA Similarity Matrix (Percentage):
            0           1           2           3           4           5   \
0   100.000000   20.000000   36.666667   23.333333   16.666667   23.333333   
1    20.000000  100.000000   16.666667   43.333333   23.333333   20.000000   
2    36.666667   16.666667  100.000000   26.666667   26.666667   43.333333   
3    23.333333   43.333333   26.666667  100.000000   26.666667   13.333333   
4    16.666667   23.333333   26.666667   26.666667  100.000000   30.000000   
5    23.333333   20.000000   43.333333   13.333333   30.000000  100.000000   
6    13.333333   33.333333   23.333333   56.666667   33.333333   10.000000   
7    33.333333   20.000000   56.666667   16.666667   30.000000   40.000000   
8    23.333333   56.666667   16.666667   50.000000   33.333333   13.333333   
9    23.333333   23.333333   63.333333   30.000000   30.000000   40.000000   
10   16.666667   30.000000   13.333333   16.666667   26.666667   13.333333   
11   20.000000   13.333

Full sibs should have about a 50% match, half would be 25%, parent child should be ~50%

In [17]:
dna[['Unnamed: 0', 'nstr']].transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
nstr,"(11,12);(30,34);(11,13);(11,12);(16,17);(7,10);(8,12);(10,10);(17,24);(13,14);(15,18);(8,10);(17,17);(7,9);(25,26)","(14,15);(28,30);(7,11);(10,11);(16,16);(7,8);(8,11);(12,12);(18,20);(13,14);(14,15);(8,8);(15,16);(9,11);(20,24)","(13,14);(30,34);(9,12);(11,12);(15,18);(9,10);(9,12);(9,11);(18,21);(13,14);(17,18);(8,11);(15,17);(12,13);(24,25)","(14,14);(30,35);(7,11);(11,11);(16,18);(6,7);(8,9);(12,13);(18,20);(14,14);(15,17);(8,11);(14,15);(9,12);(20,21)","(13,14);(28,30);(11,11);(10,11);(15,18);(6,6);(12,12);(11,11);(17,23);(12,12);(15,16);(8,9);(12,16);(12,12);(16,20)","(13,13);(30,30);(9,11);(12,13);(15,17);(7,9);(9,12);(11,11);(18,23);(13,13);(16,18);(11,12);(15,17);(11,12);(22,25)","(14,14);(29,35);(10,11);(10,11);(16,18);(6,7);(9,11);(12,13);(17,18);(14,14);(16,17);(8,11);(14,16);(12,12.0);(21,22)","(13,13);(30,34);(9,12);(10,12);(17,18);(7,10);(12,12);(9,11);(21,23);(13,14);(16,16);(8,11);(17,19);(11,11);(24,25)","(14,14);(29,30);(7,10);(10,11);(16,16);(7,7);(11,11);(12,12);(17,18);(13,14);(15,16);(8,11);(14,16);(9,12);(20,22)","(13,14);(30,31);(9,12);(10,12);(17,18);(6,7);(9,12);(9,11);(18,21);(13,14);(16,16);(8,11);(15,17);(11,11);(21,22)","(10,11);(29,29);(10,11);(10,10);(15,17);(6,8);(11,12);(12,12);(20,24);(12,14);(14,19);(8,9);(13,13);(8.0,11);(27,28)","(12,14);(29,30);(9,10);(9,12);(14,15);(6,9);(9,13);(10,13);(17,19);(14,14);(14,18);(11,11);(12,21);(11,11);(24,26)","(14,14);(28,29);(7,11);(11,11);(16,16);(7,8);(8,11);(12,12);(17,20);(14,14);(14,16);(8,11);(14,16);(11,12);(20,21)","(13,14);(28,30);(8,10);(10,12);(16,18);(6,6);(10,15);(11,13);(18,25);(13,14);(16,19);(8,12);(15,17);(7,13);(24,24)","(14,16);(29,30);(8,12);(10,11);(16,16);(6,7);(11,11);(14,14);(19,22);(13,14);(16,16);(8,11);(15,15);(10,12);(23,24)"


ndx 1 & 12 seem to be mother, son (Fort) <br>
ndx 1 & 8 seem to be mother, daughter (Fort) <br>
ndx 1 & 3 seem to be mother, son (Fort) <br>
ndx 6 seems to be the father of 3, 8, 12

ndx 2 & 5 seem to be daughter and father (hickmon) <br>
ndx 2 & 7 seem to be very related, sister and sister (Hickmon)? <br>
ndx 2 & 9 seem to be sister and brother (Hickmon) <br>
ndx 13 seems to be the mother of ndx 9 <br>
ndx 5 seems to be the father of 2, 7, 9

In [18]:
### str (14,15);(28,30);(7,11);(10,11);(16,16);(7,8);(8,11);(12,12);(18,20);(13,14);(14,15);(8,8);(15,16);(9,11);(20,24) is Jacki Fort
nstr_rm = '(14,15);(28,30);(7,11);(10,11);(16,16);(7,8);(8,11);(12,12);(18,20);(13,14);(14,15);(8,8);(15,16);(9,11);(20,24)'
fname = 'Jacki'

drop_ndx = mnfdnadf.query('nstr == @nstr_rm and firstname != @fname').index # remove occurances
mnfdnadf = mnfdnadf.drop(drop_ndx)

drop_ndx = mnfdnadf.query('nstr != @nstr_rm and firstname == @fname').index
mnfdnadf = mnfdnadf.drop(drop_ndx)

In [19]:
### str (14,14);(29,35);(10,11);(10,11);(16,18);(6,7);(9,11);(12,13);(17,18);(14,14);(16,17);(8,11);(14,16);(12,12.0);(21,22) is Miguel Fort
nstr_rm = '(14,14);(29,35);(10,11);(10,11);(16,18);(6,7);(9,11);(12,13);(17,18);(14,14);(16,17);(8,11);(14,16);(12,12.0);(21,22)'
fname = 'Miguel'

drop_ndx = mnfdnadf.query('nstr == @nstr_rm and firstname != @fname').index # remove occurances
mnfdnadf = mnfdnadf.drop(drop_ndx)

drop_ndx = mnfdnadf.query('nstr != @nstr_rm and firstname == @fname').index
mnfdnadf = mnfdnadf.drop(drop_ndx)

In [ ]:
### str (13,13);(30,30);(9,11);(12,13);(15,17);(7,9);(9,12);(11,11);(18,23);(13,13);(16,18);(11,12);(15,17);(11,12);(22,25) is Hobert Hickmon (adult)
nstr_rm = '(13,13);(30,30);(9,11);(12,13);(15,17);(7,9);(9,12);(11,11);(18,23);(13,13);(16,18);(11,12);(15,17);(11,12);(22,25)'
fname = 'Hobert'

drop_ndx = mnfdnadf.query('nstr == @nstr_rm and firstname != @fname and approx_mnf == "Adult"').index # remove occurances
mnfdnadf = mnfdnadf.drop(drop_ndx)

drop_ndx = mnfdnadf.query('nstr != @nstr_rm and firstname == @fname and approx_mnf == "Adult"').index
mnfdnadf = mnfdnadf.drop(drop_ndx)

In [21]:
### str (13,14);(28,30);(8,10);(10,12);(16,18);(6,6);(10,15);(11,13);(18,25);(13,14);(16,19);(8,12);(15,17);(7,13);(24,24) is Mari Hickmon
nstr_rm = '(13,14);(28,30);(8,10);(10,12);(16,18);(6,6);(10,15);(11,13);(18,25);(13,14);(16,19);(8,12);(15,17);(7,13);(24,24)'
mnfdnadf.query('nstr == @nstr_rm and firstname != "Mari"')

drop_ndx = mnfdnadf.query('nstr == @nstr_rm and firstname != "Mari"').index # remove occurances
mnfdnadf = mnfdnadf.drop(drop_ndx)

drop_ndx = mnfdnadf.query('nstr != @nstr_rm and firstname == "Mari"').index
mnfdnadf = mnfdnadf.drop(drop_ndx)

In [22]:
#print(len(mnfdnadf['firstname'].unique()))
#mnfdnadf

In [23]:
### Pull out duplicates
duplicate_mask = mnfdnadf.duplicated(subset=['firstname', 'lastname', 'approx_mnf'], keep=False)
dupes = mnfdnadf.loc[duplicate_mask]

In [24]:
### get pids from dupes
mnf_unmatched = temp_manif.loc[temp_manif['firstname'].isin(dupes['firstname'])]
mnf_unmatched

,pid,birth,knowndeath,firstname,lastname,gender,hoh,spouse,legaldad,legalmom,dometh,locv,loch,firstnameS,lastnameS,firstnameD,lastnameD,firstnameM,lastnameM
3,93245,7874,8200,Rick,Desalvo,M,0,NaN,NaN,NaN,0,1043.292196,1131.402808,NaN,NaN,Hipolito,Desalvo,Eustolia,Desalvo
17,101489,7744,8200,Quincy,Nally,M,0,NaN,NaN,NaN,2,1538.000000,1338.000000,NaN,NaN,NaN,NaN,NaN,NaN
19,88465,7682,8200,Shalom,Spindler,M,0,NaN,NaN,NaN,1,806.677357,696.123634,NaN,NaN,Rodrick,Spindler,Gerald,Yorke


In [25]:
### Check if the unmatched have siblings 
for row in mnf_unmatched[['firstnameD', 'lastnameD']].itertuples():
    print(row.firstnameD, row.lastnameD)
    print(dtcv.SocialQuery(E, row.firstnameD, row.lastnameD))

Hipolito Desalvo
     pid  birth  knowndeath firstname lastname gender  hoh  dometh  \
0  82776   7466        8146  Hipolito  Desalvo      M    0       3   

          locv         loch firstnameS lastnameS firstnameD lastnameD  \
0  1266.155255  1372.981271        NaN       NaN     Junior   Desalvo   

  firstnameM lastnameM  
0    Ginette   Desalvo  
nan nan
Empty DataFrame
Columns: [pid, birth, knowndeath, firstname, lastname, gender, hoh, dometh, locv, loch, firstnameS, lastnameS, firstnameD, lastnameD, firstnameM, lastnameM]
Index: []
Rodrick Spindler
     pid  birth  knowndeath firstname  lastname gender  hoh  dometh  \
0  23818   4439        5138   Rodrick  Spindler      M    0       0   
1  64494   6774        7294   Rodrick  Spindler      M    0       0   
2  76635   7232        7817   Rodrick  Spindler      M    0       0   
3  79766   7352        8155   Rodrick  Spindler      M    0       0   

          locv         loch firstnameS lastnameS  firstnameD lastnameD  \
0  1355

In [26]:
### Hipolito Desalvo children to see if there is a sibling for Rick	Desalvo

for row in dtcv.LegalChildrenQuery(E, 82776).itertuples(): #get children names
    print(dtcv.SocialQuery(E, row.firstname, row.lastname))
    

     pid  birth  knowndeath firstname     lastname gender  hoh  dometh  \
0  89120   7706          -1      Alta  Covarrubias      F    0       0   

         locv         loch firstnameS    lastnameS firstnameD lastnameD  \
0  919.255964  1321.640634   Deangelo  Covarrubias   Hipolito   Desalvo   

  firstnameM lastnameM  
0     Keesha   Desalvo  
     pid  birth  knowndeath  firstname lastname gender  hoh  dometh  \
0  89801   7732          -1  Marcellus  Desalvo      M    1       0   
1  96528   8009          -1  Marcellus  Desalvo      M    0       0   

         locv         loch firstnameS lastnameS firstnameD lastnameD  \
0  566.002712  1311.174021    Nakesha   Desalvo   Hipolito   Desalvo   
1  566.002712  1311.174021        NaN       NaN  Marcellus   Desalvo   

  firstnameM lastnameM  
0     Keesha   Desalvo  
1    Nakesha   Desalvo  
     pid  birth  knowndeath firstname lastname gender  hoh  dometh  \
0  92913   7860          -1      Tuan  Desalvo      M    1       3   

   

Tuan Desalvo seems to be brothers with Rick Desalvo, they share the same legal parents  Hipolito   Desalvo  and Eustolia   Desalvo  

In [39]:
### get tuan's dna
dtcv.DNAQuery(E,92913)

,pid,nstr,ystr,mtdna
index,,,,
92912,92913,"(12,13);(28,34);(8,13);(10,11);(15,17);(9,10);(8,12);(10,11);(22,25);(13,14);(18,19);(8,8);(16,17);(12,13);(20,24)","15;13;23;29;17.0;15;17,17;13;11;12;23;11;25;14;12;19",1;3;3;1;1;2;1;1;1;0


In [28]:
dupes.query('firstname == "Rick" and mtdna == "1;3;3;1;1;2;1;1;1;0"') 


,firstname,lastname,gender_mnf,approx_mnf,Unnamed: 0,nstr,mtdna,ystr
15,Rick,Desalvo,M,Adult,0,"(11,12);(30,34);(11,13);(11,12);(16,17);(7,10);(8,12);(10,10);(17,24);(13,14);(15,18);(8,10);(17,17);(7,9);(25,26)",1;3;3;1;1;2;1;1;1;0,"15;13;23;29;17.0;15;17,17;13;11;12;23;11;25;14;12;19"


Rick and Tuan are brothers and Hipolito   Desalvo was their father

In [29]:
### filter data again

### str (11,12);(30,34);(11,13);(11,12);(16,17);(7,10);(8,12);(10,10);(17,24);(13,14);(15,18);(8,10);(17,17);(7,9);(25,26) is Rick	Desalvo	
nstr_rm = '(11,12);(30,34);(11,13);(11,12);(16,17);(7,10);(8,12);(10,10);(17,24);(13,14);(15,18);(8,10);(17,17);(7,9);(25,26)'
fname = 'Rick'

drop_ndx = mnfdnadf.query('nstr == @nstr_rm and firstname != @fname').index # remove occurances
mnfdnadf = mnfdnadf.drop(drop_ndx)

drop_ndx = mnfdnadf.query('nstr != @nstr_rm and firstname == @fname').index
mnfdnadf = mnfdnadf.drop(drop_ndx)

In [ ]:
# Check who Rodrick Spindler, Shalom's father was married to
fname, lname = 'Rodrick', 'Spindler'
print(dtcv.WeddingQuery(E, fname, lname))

# Gerald     Dowdy       -1   8155 is likely the mother

       date   firstH     lastH   firstW     lastW  divorce  spdie
index                                                            
19991  4655  Rodrick  Spindler   Reagan     Kehoe       -1   5138
55153  6991  Rodrick  Spindler      Roy  O'Bryant       -1   7294
65436  7448  Rodrick  Spindler  Toccara      Bade       -1   7817
68212  7569  Rodrick  Spindler   Gerald     Dowdy       -1   8155


In [37]:
# Check if Rodrick Spindler, Shalom's father had any other children  

for row in dtcv.LegalChildrenQuery(E, 79766).itertuples(): #get children names
    print(row.firstname, row.lastname)
    print(dtcv.SocialQuery(E, row.firstname, row.lastname).query('birth > 7000'))

Shalom Spindler
     pid  birth  knowndeath firstname  lastname gender  hoh  dometh  \
1  88465   7682        8200    Shalom  Spindler      M    0       1   

         locv        loch firstnameS lastnameS firstnameD lastnameD  \
1  806.677357  696.123634        NaN       NaN    Rodrick  Spindler   

  firstnameM lastnameM  
1     Gerald     Yorke  
Gregory Spindler
     pid  birth  knowndeath firstname  lastname gender  hoh  dometh  \
0  89093   7705          -1   Gregory  Spindler      M    1       1   

         locv        loch firstnameS lastnameS firstnameD lastnameD  \
0  764.764343  654.829186     Evalyn  Spindler    Rodrick  Spindler   

  firstnameM lastnameM  
0     Gerald     Yorke  
Carman Loe
     pid  birth  knowndeath firstname lastname gender  hoh  dometh  \
0  89600   7724          -1    Carman      Loe      F    0       0   

         locv         loch  firstnameS lastnameS firstnameD lastnameD  \
0  661.477299  1326.913617  Christoper       Loe    Rodrick  Spindler 

Amado and Gregory Spindler are good candidates (share same legal parents)

In [41]:
### Get Amado's DNA
print(dtcv.DNAQuery(E, 94013))

### Get Gregory's DNA
print(dtcv.DNAQuery(E, 89093))

0
         pid  \
index          
89092  89093   

                                                                                                                      nstr  \
index                                                                                                                        
89092  (13,14);(28,30);(8,10);(10,12);(15,15);(6,9);(12,12);(11,11);(17,23);(12,12);(18,18);(8,11);(12,14);(12,12);(21,24)   

                                                       ystr  \
index                                                         
89092  15;13;22;29;15.0;14;17,18;13;10;12;22;11;22;14;11;21   

                     mtdna  
index                       
89092  1;1;0;2;1;2;1;0;1;1  


In [44]:
mnfdnadf.query('firstname == "Shalom" and ystr == "15;13;22;29;15.0;14;17,18;13;10;12;22;11;22;14;11;21"')

,firstname,lastname,gender_mnf,approx_mnf,Unnamed: 0,nstr,mtdna,ystr
214,Shalom,Spindler,M,Adult,4,"(13,14);(28,30);(11,11);(10,11);(15,18);(6,6);(12,12);(11,11);(17,23);(12,12);(15,16);(8,9);(12,16);(12,12);(16,20)",1;1;0;2;1;2;1;0;1;1,"15;13;22;29;15.0;14;17,18;13;10;12;22;11;22;14;11;21"


Shalom and Gregory share the same legal dad and legal mom, have the same mtdna and ystr, we found a match!

In [45]:
### filter data again

### str (13,14);(28,30);(11,11);(10,11);(15,18);(6,6);(12,12);(11,11);(17,23);(12,12);(15,16);(8,9);(12,16);(12,12);(16,20) is Shalom	Spindler
nstr_rm = '(13,14);(28,30);(11,11);(10,11);(15,18);(6,6);(12,12);(11,11);(17,23);(12,12);(15,16);(8,9);(12,16);(12,12);(16,20)'
fname = 'Shalom'

drop_ndx = mnfdnadf.query('nstr == @nstr_rm and firstname != @fname').index # remove occurances
mnfdnadf = mnfdnadf.drop(drop_ndx)

drop_ndx = mnfdnadf.query('nstr != @nstr_rm and firstname == @fname').index
mnfdnadf = mnfdnadf.drop(drop_ndx)

In [48]:
mnfdnadf

,firstname,lastname,gender_mnf,approx_mnf,Unnamed: 0,nstr,mtdna,ystr
11,Wanetta,Brey,F,Adult,11,"(12,14);(29,30);(9,10);(9,12);(14,15);(6,9);(9,13);(10,13);(17,19);(14,14);(14,18);(11,11);(12,21);(11,11);(24,26)",1;3;w1;1;2;2;4;1;1;1,NaN
15,Rick,Desalvo,M,Adult,0,"(11,12);(30,34);(11,13);(11,12);(16,17);(7,10);(8,12);(10,10);(17,24);(13,14);(15,18);(8,10);(17,17);(7,9);(25,26)",1;3;3;1;1;2;1;1;1;0,"15;13;23;29;17.0;15;17,17;13;11;12;23;11;25;14;12;19"
42,Arlie,Fort,M,Young Adult,12,"(14,14);(28,29);(7,11);(11,11);(16,16);(7,8);(8,11);(12,12);(17,20);(14,14);(14,16);(8,11);(14,16);(11,12);(20,21)",1;2;1;1;2;2;w2;1;1;1,"16;13;24;29;18.0;14;15,18;13;10;12;22;13;23;14;12;20"
53,Carmella,Fort,F,Adolescent,8,"(14,14);(29,30);(7,10);(10,11);(16,16);(7,7);(11,11);(12,12);(17,18);(13,14);(15,16);(8,11);(14,16);(9,12);(20,22)",1;2;1;1;2;2;w2;1;1;1,NaN
61,Jacki,Fort,F,Adult,1,"(14,15);(28,30);(7,11);(10,11);(16,16);(7,8);(8,11);(12,12);(18,20);(13,14);(14,15);(8,8);(15,16);(9,11);(20,24)",1;2;1;1;2;2;w2;1;1;1,NaN
81,Miguel,Fort,M,Adult,6,"(14,14);(29,35);(10,11);(10,11);(16,18);(6,7);(9,11);(12,13);(17,18);(14,14);(16,17);(8,11);(14,16);(12,12.0);(21,22)",1;1;4;1;1;2;5;1;1;1,"16;13;24;29;18.0;14;15,18;13;10;12;22;13;23;14;12;20"
93,Nelson,Fort,M,Adolescent,3,"(14,14);(30,35);(7,11);(11,11);(16,18);(6,7);(8,9);(12,13);(18,20);(14,14);(15,17);(8,11);(14,15);(9,12);(20,21)",1;2;1;1;2;2;w2;1;1;1,"16;13;24;29;18.0;14;15,18;13;10;12;22;13;23;14;12;20"
112,Anneliese,Hickmon,F,Adolescent,7,"(13,13);(30,34);(9,12);(10,12);(17,18);(7,10);(12,12);(9,11);(21,23);(13,14);(16,16);(8,11);(17,19);(11,11);(24,25)",1;1;3;1;1;2;0;1;1;1,NaN
125,Hobert,Hickmon,M,Adult,5,"(13,13);(30,30);(9,11);(12,13);(15,17);(7,9);(9,12);(11,11);(18,23);(13,13);(16,18);(11,12);(15,17);(11,12);(22,25)",1;0;0;2;2;2;3;1;1;1,"16;12;24;29;17.0;15;18,18;13;11;11;23;13;22;16;12;20,21"
144,Hobert,Hickmon,M,Adolescent,9,"(13,14);(30,31);(9,12);(10,12);(17,18);(6,7);(9,12);(9,11);(18,21);(13,14);(16,16);(8,11);(15,17);(11,11);(21,22)",1;1;3;1;1;2;0;1;1;1,"16;12;24;29;17.0;15;18,18;13;11;11;23;13;22;16;12;20,21"
